In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
folder=r"C:\Users\Rajath\Desktop\Face-Mask-Detection-master\dataset\train"
cat=["mask", "no_mask"]


In [3]:
x = []
y = []

for ca in cat:
    path = os.path.join(folder, ca)
    for im in os.listdir(path):
    	im_path = os.path.join(path, im)
    	img = load_img(im_path, target_size=(224, 224))
    	img = img_to_array(img)
    	img = preprocess_input(img)

    	x.append(img)
    	y.append(ca)

lb = LabelBinarizer()
y = lb.fit_transform(y)
y = to_categorical(y)

E:\anacond3_9\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [4]:
x = np.array(x, dtype="float32")
y = np.array(y)

(x_train, x_test, y_train, y_test) = train_test_split(x, y,test_size=0.20, stratify=y, random_state=42)

# construct the training image generator for data augmentation
augment = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [5]:
from keras.applications import VGG16
bm = VGG16(weights = 'imagenet',  include_top = False, input_shape = (224, 224, 3))



In [6]:
hm = bm.output
hm = AveragePooling2D(pool_size=(7, 7))(hm)
hm = Flatten(name="flatten")(hm)
hm = Dense(128, activation="relu")(hm)
hm = Dropout(0.5)(hm)
hm = Dense(2, activation="softmax")(hm)

In [7]:
vgg_model = Model(inputs=bm.input, outputs=hm)
for layer in bm.layers:
    layer.trainable = False
    
optimiser = Adam(lr=0.0001, decay=0.0001 / 20)
vgg_model.compile(loss="binary_crossentropy", optimizer=optimiser,metrics=["accuracy"])

E:\anacond3_9\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
h = vgg_model.fit(augment.flow(x_train, y_train, batch_size=32),steps_per_epoch=len(x_train) // 32,validation_data=(x_test, y_test),validation_steps=len(x_test) // 32,epochs=7)


Epoch 1/7
116/116 [==============================] - 1300s 11s/step - loss: 0.6810 - accuracy: 0.6142 - val_loss: 0.5867 - val_accuracy: 0.7310
Epoch 2/7
116/116 [==============================] - 981s 8s/step - loss: 0.5883 - accuracy: 0.7300 - val_loss: 0.5022 - val_accuracy: 0.8875
Epoch 3/7
116/116 [==============================] - 993s 9s/step - loss: 0.5032 - accuracy: 0.8404 - val_loss: 0.4253 - val_accuracy: 0.8832
Epoch 4/7
116/116 [==============================] - 987s 9s/step - loss: 0.4352 - accuracy: 0.8791 - val_loss: 0.3656 - val_accuracy: 0.8960
Epoch 5/7
116/116 [==============================] - 990s 9s/step - loss: 0.3813 - accuracy: 0.8948 - val_loss: 0.3162 - val_accuracy: 0.9121
Epoch 6/7
116/116 [==============================] - 985s 9s/step - loss: 0.3438 - accuracy: 0.8996 - val_loss: 0.2728 - val_accuracy: 0.9260
Epoch 7/7
116/116 [==============================] - 984s 8s/step - loss: 0.3118 - accuracy: 0.9061 - val_loss: 0.2458 - val_accuracy: 0.9271


In [9]:
pre = vgg_model.predict(x_test,batch_size=32)
print(pre)

#pre = np.argmax(pre, axis=1)

#print(classification_report(y_test.argmax(axis=1), pre,target_names=lb.classes_))


30/30 [==============================] - 198s 7s/step
[[0.65455776 0.3454423 ]
 [0.35165533 0.64834464]
 [0.13911591 0.86088413]
 ...
 [0.00514391 0.9948561 ]
 [0.03330072 0.96669924]
 [0.99647826 0.00352173]]


In [10]:
vgg_model.save("vgg16.model", save_format="h5")

In [11]:
print(pre)

[[0.65455776 0.3454423 ]
 [0.35165533 0.64834464]
 [0.13911591 0.86088413]
 ...
 [0.00514391 0.9948561 ]
 [0.03330072 0.96669924]
 [0.99647826 0.00352173]]
